In [ ]:
import pandas as pd
import numpy as np
import pydeck as pdk



color_range = [
    [29,53,87,220],
    [69,123,157,220],
    [168,218,220,220],
    [241,250,238,220],
    [239,35,60,220],
    [217,4,41,220]
]

veg_df = pd.read_csv(r'GEO_Dysart_20210408_grow15_fall5.csv')
veg_df.loc[(veg_df['vegetation_height']>=8) & (veg_df['vegetation_height']<=10)]

# max_val = veg_df.loc[:,'vegetation_height'].max(axis=0)
# min_val = veg_df.loc[:,'vegetation_height'].min(axis=0)


# radius_scale = 1
# radius_unit =1
# veg_df.loc[:,'normValue'] = ((veg_df.loc[:,'vegetation_height']/(max_val-min_val))*radius_unit*radius_scale)

# veg_df.loc[:,'fillColorIndex'] = ((veg_df.loc[:,'vegetation_height']-min_val) / (max_val-min_val) )*(len(color_range) - 1)
# veg_df.loc[:,'fill_color'] = veg_df.loc[:,'fillColorIndex'].map(lambda x: color_range[int(x)])

# scatterplot = pdk.Layer(
#     type='ScatterplotLayer',
#     id='scatterplot-layer',
#     data=veg_df,
#     pickable=True,
#     get_position=['longitude', 'latitude'],
#     get_radius='normValue',
#     radius_min_pixels=2*radius_scale,
#     radius_max_pixels=30*radius_scale,
#     get_color='fill_color',
#     stroked=True,
#     filled=True,
#     opacity=0
# )

# map = pdk.Deck(scatterplot, map_style = 'mapbox://styles/mapbox/light-v9',

#         initial_view_state=pdk.ViewState(
#             latitude=33.90578688,
#             longitude=-116.9083177,
#             zoom=20,
#             pitch=0,
#             ),
#         tooltip={"html": "<b>Tree Height: </b> {vegetation_height} ft<br><b>Conductor Height: </b> {h_conductor} ft</br>", "style": {"color": "white"}}
#     )
# map
def hex_to_RGB(hex):
    ''' "#FFFFFF" -> [255,255,255] '''
    # Pass 16 to the integer function for change of base
    return [int(hex[i:i+2], 16) for i in range(1,6,2)]

def linear_gradient(start_hex, finish_hex="#FFFFFF", n=len(veg_df['vegetation_height'])):
    ''' returns a gradient list of (n) colors between
    two hex colors. start_hex and finish_hex
    should be the full six-digit color string,
    including the number sign ("#FFFFFF") '''
    # Starting and ending colors in RGB form
    s = hex_to_RGB(start_hex)
    f = hex_to_RGB(finish_hex)
    # Initilize a list of the output colors with the starting color
    RGB_list = [s]
    # Calcuate a color at each evenly spaced value of t from 1 to n
    for t in range(1, n):
        # Interpolate RGB vector for color at the current value of t
        curr_vector = [
        int(s[j] + (float(t)/(n-1))*(f[j]-s[j]))
        for j in range(3)
        ]
        # Add it to our list of output colors
        RGB_list.append(curr_vector)

    return RGB_list

veg_df = veg_df.sort_values(by=['vegetation_height'])

df = pd.DataFrame(linear_gradient("#4682B4", "#FFB347"), columns=['r','g','b'])
veg_df[['r','g','b']] = 0
veg_df['r','g','b'] = df

veg_df


In [ ]:
import pandas as pd
import pydeck as pdk
import streamlit as st
import numpy as np

veg_df = pd.read_csv(r'Data/GEO_Dysart_20210408_grow15_fall5.csv')
df = pd.DataFrame(np.random.randn(1000, 2) / [50, 50] + [37.76, -122.4],columns=['lat', 'lon'])

veg_df = veg_df[['longitude', 'latitude', 'vegetation_height']]
pdk.Deck(
    map_style='mapbox://styles/mapbox/light-v9',
    initial_view_state=pdk.ViewState(
        latitude=33.90578688,
        longitude=-116.9083177,
        zoom=11,
        pitch=50,
        ),
        layers=[
            pdk.Layer(
            'HexagonLayer',
            data=df,
            get_position='[lon, lat]',
            radius=200,
            elevation_scale=4,
            elevation_range=[0, 1000],
            pickable=True,
            extruded=True,
        ),
        pdk.Layer(
            'ScatterplotLayer',
            data=veg_df,
            get_position='[longitude, latitude]',
            get_color='[200, 30, 0, 160]',
            get_radius=50,
            pickable=True,
        ),
    ],
)



In [17]:
import pandas as pd

def hex_to_RGB(hex):
    ''' "#FFFFFF" -> [255,255,255] '''
    # Pass 16 to the integer function for change of base
    return [int(hex[i:i+2], 16) for i in range(1,6,2)]

def linear_gradient(df_length, start_hex, finish_hex="#FFFFFF"):
    ''' returns a gradient list of (n) colors between
    two hex colors. start_hex and finish_hex
    should be the full six-digit color string,
    including the number sign ("#FFFFFF") '''
    # Starting and ending colors in RGB form
    s = hex_to_RGB(start_hex)
    f = hex_to_RGB(finish_hex)
    # Initilize a list of the output colors with the starting color
    RGB_list = [s]
    # Calcuate a color at each evenly spaced value of t from 1 to n
    for t in range(1, df_length):
        # Interpolate RGB vector for color at the current value of t
        curr_vector = [
        int(s[j] + (float(t)/(df_length-1))*(f[j]-s[j]))
        for j in range(3)
        ]
        # Add it to our list of output colors
        RGB_list.append(curr_vector)
    return RGB_list

df = pd.read_csv(r'GEO_Dysart_20210408_grow15_fall5.csv')
conductor_height_slider = [38, 40]
df = df.sort_values(by=['vegetation_height'])
color_df = pd.DataFrame(linear_gradient(len(df), "#4682B4", "#FFB347"), columns=['r','g','b'])
df = pd.concat([df, color_df], axis=1)

filtered_df = df[(df['h_conductor']>=conductor_height_slider[0]) & (df['h_conductor']<=conductor_height_slider[1])]
filtered_df = filtered_df.sort_values(by=['vegetation_height'])
filtered_df = filtered_df.reset_index()
##### FIX #####
color_df = pd.DataFrame(linear_gradient(len(filtered_df), "#4682B4", "#FFB347"), columns=['r','g','b'])
filtered_df[['r','g','b']] = color_df
filtered_df = filtered_df.drop(columns=['r','g','b'])
filtered_df = pd.concat([filtered_df, color_df], axis = 1)
filtered_df




,index,vegetation_id,block_id,hazard_condition,status,voltage,h_conductor,x_distance,pole_from,pole_to,...,latitude,las_id,F3d_clearance,vegetation_height,floc_id,circuit_name,flight_date,r,g,b
0,1966,548301967,NaN,Grow,GRCD: >=12ft,12KV,38.13,3.80,4467341E,2138061E,...,33.906991,Dysart-GEO1-20210408,14.67,23.43,ED-05483,DYSART,20210408,70,130,180
1,1932,548301933,NaN,Grow,GRCD: >=12ft,12KV,38.30,3.98,2138064E,2138063E,...,33.906986,Dysart-GEO1-20210408,14.40,23.45,ED-05483,DYSART,20210408,71,130,179
2,1852,548301853,NaN,Grow,GRCD: >=12ft,12KV,38.32,3.08,4016233E,4016234E,...,33.899915,Dysart-GEO1-20210408,14.00,23.69,ED-05483,DYSART,20210408,73,130,178
3,1986,548301987,NaN,Grow,GRCD: >=12ft,12KV,38.93,1.60,2138060E,2138059E,...,33.906971,Dysart-GEO1-20210408,14.77,23.82,ED-05483,DYSART,20210408,74,131,177
4,1990,548301991,NaN,Grow,GRCD: >=12ft,12KV,38.83,1.16,2210868E,2210867E,...,33.917536,Dysart-GEO1-20210408,14.80,24.07,ED-05483,DYSART,20210408,76,131,176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,27,548300028,NaN,Fall,Fall-in <= -20ft,12KV,39.20,17.75,2293466E,2293465E,...,33.905586,Dysart-GEO1-20210408,-27.48,71.08,ED-05483,DYSART,20210408,248,177,74
115,49,548300050,NaN,Fall,Fall-in <= -20ft,12KV,39.07,27.91,2293466E,2293465E,...,33.905592,Dysart-GEO1-20210408,-24.24,72.90,ED-05483,DYSART,20210408,250,177,73
116,8,548300009,NaN,Fall,Fall-in <= -20ft,12KV,38.55,13.95,2293466E,2293465E,...,33.905612,Dysart-GEO1-20210408,-32.27,73.66,ED-05483,DYSART,20210408,251,178,72
117,84,548300085,NaN,Fall,Fall-in <= -20ft,12KV,38.95,33.43,2138064E,2138063E,...,33.906888,Dysart-GEO1-20210408,-20.61,74.94,ED-05483,DYSART,20210408,253,178,71


In [18]:
import pandas as pd
import plotly as plt

df = pd.read_csv(r'GEO_Dysart_20210408_grow15_fall5.csv')



<AxesSubplot:>